### 1. Install dependencies

In [126]:
import os
from dotenv import load_dotenv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By


### 2. FX scraper

In [129]:
url = "https://www.dbs.com/in/treasures/rates-online/foreign-currency-foreign-exchange.page"

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get(url)

In [132]:
date_detail = driver.find_element(By.ID, "mainCurrency").text
elements = date_detail.split(' ')

date = elements[3]
time = elements[-1]

In [3]:
# Header Names
header = driver.find_elements(By.XPATH, "//table[@class='sc-dxgOiQ dzHkKC tbl-primary mBot-16']/thead/tr/th")

col_names = list()

for head in header:
    col_names.append(head.text)

print(col_names)


['Currency', 'Selling TT', 'Selling Cash', 'Buying TT', 'Buying Cash']


In [4]:
# Locators
col = driver.find_elements(By.XPATH, "//table[@class='sc-dxgOiQ dzHkKC tbl-primary mBot-16']/tbody/tr")
row = driver.find_elements(By.XPATH, "//table[@class='sc-dxgOiQ dzHkKC tbl-primary mBot-16']/tbody/tr[1]/td")

num_row = len(row) #5
num_col = len(col) #10

results = []


for i in range (1, num_col + 1):
    result = []
    for j in range (1, num_row + 1):
        d = driver.find_element(By.XPATH, "//table[@class='sc-dxgOiQ dzHkKC tbl-primary mBot-16']/tbody/tr["+str(i)+"]/td["+str(j)+"]").text
        result.append(d)
    results.append(result)

In [6]:
df = pd.DataFrame(data=results, columns=col_names)

df["Date"] = date
df["Time"] = time

print(df)

                      Currency Selling TT Selling Cash Buying TT Buying Cash  \
0                    US Dollar    81.2468      81.2468   78.0607     78.0607   
1             Singapore Dollar    59.4800      59.4800   56.8625     56.8625   
2            Australian Dollar    57.9094      57.9094   55.3610     55.3610   
3                  Swiss Franc    86.5179      86.5179   82.7106     82.7106   
4                         Euro    84.0371      84.0371   80.3389     80.3389   
5                British Pound    99.3098      99.3098   94.9394     94.9394   
6              Canadian Dollar    63.8483      63.8483   61.0385     61.0385   
7             Hong Kong Dollar    10.3947      10.3947    9.9373      9.9373   
8                 Japanese Yen     0.6111       0.6111    0.5842      0.5842   
9  United Arab Emirates Dirham    22.1744      22.1744   21.1986     21.1986   

         Date     Time  
0  12/08/2022  11:45AM  
1  12/08/2022  11:45AM  
2  12/08/2022  11:45AM  
3  12/08/2022  11:4

#### 2. PDF File

In [115]:
pdf = "/home/oem/Documents/coding/personal/Reits-Analytics/data/Ascendas-Reit-Annual-Report-2021.pdf"

import tabula
import pandas as pd

tables = tabula.read_pdf(pdf, pages=[72])
df = tables[0]
df = df.rename(columns={"Unnamed: 0": "num", 
                "Unnamed: 1": "property", 
                "Business & Science Park Properties": "development_cost",	
                "Unnamed: 2": "valuations_(SGD:Mil)",
                "Unnamed: 3": "gfa_(Sqm)",
                "Unnamed: 4": "nla_(Sqm)",
                "Unnamed: 5": "address",
                "Unnamed: 6": "gross_revenue_(SGD:Mil)",
                "Unnamed: 7": "occupancy_rate"})
# remove first rows
df = df.iloc[4:]
# split out completion date parsed incorrectly
def split_completion_date(text):
    if isinstance(text, str):
        x = text[:9]
        return x

df["completion_date"] = df["development_cost"].apply(lambda x: split_completion_date(x))
# split out development cost parsed incorrectly
def split_development_cost(text):
    if isinstance(text, str) and len(text) > 11:
        amt = text[10:]
        return amt

df["development_cost"] = df["development_cost"].apply(lambda x: split_development_cost(x))
df = df.dropna(how="any", subset=["development_cost", "completion_date"])

df

3. Geographical API

In [ ]:


path = "https://maps.googleapis.com/maps/api/geocode/json?address=1600+Amphitheatre+Parkway,+Mountain+View,+CA&key=YOUR_API_KEY"